In [1]:
import sys
print(sys.executable)


C:\projects\vertex_ai_course\.venv\Scripts\python.exe


In [1]:
from kfp import dsl
from kfp import compiler


import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="kfp.*")

In [2]:
@dsl.component
def say_hello(name: str) -> str:
    hello_text = f"Hello, {name}!"
    return hello_text


In [3]:
hello_task = say_hello(name="Erwin")
print(hello_task)
print(hello_task.output)

{{channel:task=say-hello;name=Output;type=String;}}


In [4]:
@dsl.component
def how_are_you(hello_text: str) -> str:
    how_are_you = f"{hello_text}. How are you?"
    return how_are_you

In [5]:
how_task = how_are_you(hello_text=hello_task.output)
print(how_task)
print(how_task.output)

{{channel:task=how-are-you;name=Output;type=String;}}


In [6]:
@dsl.pipeline
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    how_task = how_are_you(hello_text=hello_task.output)
    return how_task.output


In [7]:
compiler.Compiler().compile(hello_pipeline, "pipeline.yaml")


In [8]:
!type pipeline.yaml


# PIPELINE DEFINITION
# Name: hello-pipeline
# Inputs:
#    recipient: str
# Outputs:
#    Output: str
components:
  comp-how-are-you:
    executorLabel: exec-how-are-you
    inputDefinitions:
      parameters:
        hello_text:
          parameterType: STRING
    outputDefinitions:
      parameters:
        Output:
          parameterType: STRING
  comp-say-hello:
    executorLabel: exec-say-hello
    inputDefinitions:
      parameters:
        name:
          parameterType: STRING
    outputDefinitions:
      parameters:
        Output:
          parameterType: STRING
deploymentSpec:
  executors:
    exec-how-are-you:
      container:
        args:
        - --executor_input
        - '{{$}}'
        - --function_to_execute
        - how_are_you
        command:
        - sh
        - -c
        - "\nif ! [ -x \"$(command -v pip)\" ]; then\n    python3 -m ensurepip ||\
          \ python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1\
   

In [9]:
PROJECT_ID = "dotted-music-460617-k2"
REGION = "us-central1"

BUCKET_NAME = "krishna-dotted-music-460617-k2-vertex-pipeline-bucket"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root"
print("PIPELINE_ROOT =", PIPELINE_ROOT)


PIPELINE_ROOT = gs://krishna-dotted-music-460617-k2-vertex-pipeline-bucket/pipeline_root


In [10]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)


In [12]:
job = aiplatform.PipelineJob(
    display_name="hello-pipeline-run",
    template_path="pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"recipient": "Vamshi"},
)

job.run(sync=True)  # IMPORTANT: sync=True for debugging
print("RESOURCE NAME:", job.resource_name)
print("STATE:", job.state)


Creating PipelineJob
PipelineJob created. Resource name: projects/864952758295/locations/us-central1/pipelineJobs/hello-pipeline-20251217110702
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/864952758295/locations/us-central1/pipelineJobs/hello-pipeline-20251217110702')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/hello-pipeline-20251217110702?project=864952758295
PipelineJob projects/864952758295/locations/us-central1/pipelineJobs/hello-pipeline-20251217110702 current state:
3
PipelineJob projects/864952758295/locations/us-central1/pipelineJobs/hello-pipeline-20251217110702 current state:
3
PipelineJob projects/864952758295/locations/us-central1/pipelineJobs/hello-pipeline-20251217110702 current state:
3
PipelineJob projects/864952758295/locations/us-central1/pipelineJobs/hello-pipeline-20251217110702 current state:
3
PipelineJob projects/864952758295/locations/us-central1/pipelineJo

### these are the same 
### jsonl files from the previous lab

### time stamps have been removed so that 
### the files are consistent for all learners
TRAINING_DATA_URI = "./tune_data_stack_overflow_python_qa.jsonl" 
EVAUATION_DATA_URI = "./tune_eval_data_stack_overflow_python_qa.jsonl"  


from google.cloud import aiplatform
from utils import authenticate

credentials, PROJECT_ID = authenticate()
REGION = "us-central1"

aiplatform.init(project=PROJECT_ID, location=REGION, credentials=credentials)

BUCKET = "krishna-dotted-music-460617-k2-vertex-pipeline-bucket"
PIPELINE_ROOT = f"gs://{BUCKET}/pipeline-root"


TRAINING_DATA_URI = f"gs://{BUCKET}/datasets/tune_data_stack_overflow_python_qa.jsonl"
EVALUATION_DATA_URI = f"gs://{BUCKET}/datasets/tune_eval_data_stack_overflow_python_qa.jsonl"


import datetime
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
MODEL_NAME = f"deep-learning-ai-model-{date}"

TRAINING_STEPS = 200
EVALUATION_INTERVAL = 20

pipeline_arguments = {
    "model_display_name": MODEL_NAME,
    "location": REGION,
    "large_model_reference": "text-bison@001",
    "project": PROJECT_ID,
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": EVALUATION_INTERVAL,
    "evaluation_data_uri": EVALUATION_DATA_URI,
}


template_path = "https://us-kfp.pkg.dev/ml-pipeline/large-language-model-pipelines/tune-large-model/v2.0.0"

job = aiplatform.PipelineJob(
    display_name=f"tune-llm-pipeline-{date}",
    template_path=template_path,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=pipeline_arguments,
    enable_caching=True,
)

job.submit()
print("Submitted:", job.resource_name)


---
## ⚠️ NOTE: Course Code Above is DEPRECATED

The course used `text-bison@001` which Google deprecated in 2024.

Below is the **updated code using Gemini** (the current model).

---

!pip install --upgrade google-cloud-aiplatform


# ============================================================================
# Gemini Fine-Tuning - Using v1beta1 API
# ============================================================================

from google.cloud import aiplatform
import datetime

# Configuration
PROJECT_ID = "dotted-music-460617-k2"
REGION = "us-central1"
BUCKET = "krishna-dotted-music-460617-k2-vertex-pipeline-bucket"

# Initialize
print("🚀 Initializing Vertex AI...")
aiplatform.init(project=PROJECT_ID, location=REGION)

# Data paths
TRAIN_GCS = f"gs://{BUCKET}/datasets/tune_data_stack_overflow_python_qa.jsonl"
EVAL_GCS = f"gs://{BUCKET}/datasets/tune_eval_stack_overflow_python_qa.jsonl"

# Model name
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
MODEL_NAME = f"stackoverflow-gemini-{date}"

print(f"📝 Model Name: {MODEL_NAME}")
print(f"📂 Training Data: {TRAIN_GCS}")
print(f"📂 Validation Data: {EVAL_GCS}")

# Use v1beta1 (not v1)
print("\n⏳ Starting Gemini fine-tuning...")

from google.cloud.aiplatform_v1beta1 import GenAiTuningServiceClient  # ← Changed to v1beta1
from google.cloud.aiplatform_v1beta1.types import SupervisedTuningSpec, TuningJob as TuningJobProto

# Get the client
client = GenAiTuningServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

# Create tuning job
parent = f"projects/{PROJECT_ID}/locations/{REGION}"

tuning_job = TuningJobProto(
    base_model="gemini-1.5-flash-002",
    supervised_tuning_spec=SupervisedTuningSpec(
        training_dataset_uri=TRAIN_GCS,
        validation_dataset_uri=EVAL_GCS,
        hyper_parameters=SupervisedTuningSpec.HyperParameters(
            epoch_count=3,
            learning_rate_multiplier=1.0,
        )
    ),
    tuned_model_display_name=MODEL_NAME,
)

print("   Submitting job to Google Cloud...")
response = client.create_tuning_job(parent=parent, tuning_job=tuning_job)

print("\n✅ SUCCESS! Training job submitted")
print(f"\n📍 Job Name:")
print(f"   {response.name}")

print(f"\n🌐 Monitor Progress:")
print(f"   https://console.cloud.google.com/vertex-ai/generative/language/tuning/train?project={PROJECT_ID}")

# Save job name
with open('tuning_job_name.txt', 'w') as f:
    f.write(response.name)
    
print("\n✓ Job info saved to: tuning_job_name.txt")
print("\n⏳ Training will take 20-40 minutes")
print("💡 You can close this notebook - training runs in the cloud!")

In [43]:
# ============================================================================
# DIAGNOSTIC: Find the Correct Import Path
# ============================================================================

import google.cloud.aiplatform_v1beta1 as v1beta1

print("=" * 70)
print("PART 1: CHECKING v1beta1 MODULE")
print("=" * 70)

# Check what's in the main module
available = dir(v1beta1)
print(f"\nTotal items in v1beta1: {len(available)}")

# Look for anything tuning-related
tuning_related = [item for item in available if 'tuning' in item.lower() or 'genai' in item.lower()]
print(f"\nTuning-related items found: {len(tuning_related)}")
for item in tuning_related:
    print(f"  ✓ {item}")

print("\n" + "=" * 70)
print("PART 2: CHECKING SERVICES MODULE")
print("=" * 70)

try:
    from google.cloud.aiplatform_v1beta1 import services
    services_list = dir(services)
    print(f"\nTotal services: {len(services_list)}")
    
    # Filter for tuning
    tuning_services = [s for s in services_list if 'tuning' in s.lower() or 'genai' in s.lower() or 'gen_ai' in s.lower()]
    print(f"\nTuning-related services found: {len(tuning_services)}")
    for service in tuning_services:
        print(f"  ✓ {service}")
        
        # Try to import it
        try:
            module = getattr(services, service)
            print(f"    → Can import: {module}")
        except Exception as e:
            print(f"    → Import error: {e}")
            
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70)
print("PART 3: CHECKING TYPES MODULE")
print("=" * 70)

try:
    from google.cloud.aiplatform_v1beta1 import types
    types_list = dir(types)
    
    tuning_types = [t for t in types_list if 'tuning' in t.lower() or 'supervised' in t.lower()]
    print(f"\nTuning-related types found: {len(tuning_types)}")
    for t in tuning_types[:10]:  # Show first 10
        print(f"  ✓ {t}")
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70)
print("PART 4: CHECK PACKAGE VERSION INFO")
print("=" * 70)

import google.cloud.aiplatform
print(f"\nPackage version: {google.cloud.aiplatform.__version__}")
print(f"Package location: {google.cloud.aiplatform.__file__}")

print("\n" + "=" * 70)
print("DIAGNOSTIC COMPLETE")
print("=" * 70)

PART 1: CHECKING v1beta1 MODULE

Total items in v1beta1: 697

Tuning-related items found: 8
  ✓ CancelHyperparameterTuningJobRequest
  ✓ CreateHyperparameterTuningJobRequest
  ✓ DeleteHyperparameterTuningJobRequest
  ✓ GetHyperparameterTuningJobRequest
  ✓ HyperparameterTuningJob
  ✓ ListHyperparameterTuningJobsRequest
  ✓ ListHyperparameterTuningJobsResponse
  ✓ ManualBatchTuningParameters

PART 2: CHECKING SERVICES MODULE

Total services: 34

Tuning-related services found: 1
  ✓ gen_ai_tuning_service
    → Can import: <module 'google.cloud.aiplatform_v1beta1.services.gen_ai_tuning_service' from 'C:\\projects\\vertex_ai_course\\.venv\\Lib\\site-packages\\google\\cloud\\aiplatform_v1beta1\\services\\gen_ai_tuning_service\\__init__.py'>

PART 3: CHECKING TYPES MODULE

Tuning-related types found: 12
  ✓ CancelHyperparameterTuningJobRequest
  ✓ CreateHyperparameterTuningJobRequest
  ✓ DeleteHyperparameterTuningJobRequest
  ✓ GetHyperparameterTuningJobRequest
  ✓ HyperparameterTuningJob
  

# ============================================================================
# Find Available Models for Tuning
# ============================================================================

from google.cloud import aiplatform
from google.cloud.aiplatform_v1beta1.services.gen_ai_tuning_service import GenAiTuningServiceClient

PROJECT_ID = "dotted-music-460617-k2"
REGION = "us-central1"

# Initialize
aiplatform.init(project=PROJECT_ID, location=REGION)

# Create client
client = GenAiTuningServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

print("=" * 70)
print("CHECKING AVAILABLE BASE MODELS FOR TUNING")
print("=" * 70)

# Try to list tuning jobs to see what models are being used
parent = f"projects/{PROJECT_ID}/locations/{REGION}"

try:
    # List existing tuning jobs to see what models are used
    request = {"parent": parent}
    page_result = client.list_tuning_jobs(request=request)
    
    print("\nExisting tuning jobs in your project:")
    for response in page_result:
        print(f"  Model: {response.base_model}")
        print(f"  Name: {response.name}")
        print(f"  State: {response.state}")
        print("-" * 70)
        
except Exception as e:
    print(f"No existing jobs or error: {e}")

print("\n" + "=" * 70)
print("COMMON MODEL NAMES FOR TUNING:")
print("=" * 70)
print("""
Try these model names:
1. text-bison@002 (newer Bison, might still work)
2. gemini-1.0-pro-002
3. gemini-1.5-pro-002  
4. gemini-1.5-flash-001
5. publishers/google/models/gemini-1.5-flash-002

Let's test which one works...
""")

# Test which model name works
test_models = [
    "text-bison@002",
    "gemini-1.0-pro-002",
    "gemini-1.5-pro-002",
    "gemini-1.5-flash-001",
    "publishers/google/models/gemini-1.5-flash-002",
    "gemini-1.5-flash-002",
]

for model_name in test_models:
    print(f"\nTesting: {model_name}")
    try:
        from google.cloud.aiplatform_v1beta1.types import TuningJob, SupervisedTuningSpec
        
        tuning_job = TuningJob(
            base_model=model_name,
            supervised_tuning_spec=SupervisedTuningSpec(
                training_dataset_uri=TRAIN_GCS,
                validation_dataset_uri=EVAL_GCS,
                hyper_parameters={"epoch_count": 3, "learning_rate_multiplier": 1.0}
            ),
            tuned_model_display_name=f"test-{model_name.replace('/', '-')}",
        )
        
        # Try to submit (will validate model name)
        response = client.create_tuning_job(parent=parent, tuning_job=tuning_job)
        print(f"  ✅ SUCCESS! {model_name} works!")
        print(f"  Job: {response.name}")
        break  # Stop on first success
        
    except Exception as e:
        error_msg = str(e)
        if "not supported" in error_msg:
            print(f"  ❌ Not supported")
        else:
            print(f"  ❌ Error: {error_msg[:100]}")